## TS3 - TP2 Ejercicio 4

A partir de la siguiente plantilla, sabiendo que:

![Plantilla](img/plantilla_original.JPG)

$\alpha_{Máx} = 1 dB$

$\alpha_{Min} = 35 dB$

$f_p = 3500 Hz$

$f_s = 1000 Hz$

* Obtener polos y ceros para máxima planicidad en la banda de paso.
* Implementar el circuito con estructuras pasivas adaptadas mediante buffers.
* Utilizando una norma de impedancia $\Omega_Z = 1K\Omega$, obtenga el valor de los componentes.
* Active las bobinas utilizando una estructura con OPAMPs.

Primero debemos transformar la plantilla del pasa-altos a una plantilla para el pasa-bajos prototipo, que es el filtro que sabemos trabajar. Además, normalizamos la plantilla en frecuencia, tomando como norma de frecuencia $\Omega_{\omega} = 2\pi f_p$

<div><img src="img/plantilla_transformada.jpeg" width="500"/></div>

A continuación, planteamos la función de aproximación por máxima planicidad para el pasabajos:

$|T_{(j\Omega)}|^2=\frac{1}{1+\epsilon^2 . \Omega^{2 . n}}$

Llamamos $\Omega$ a la frecuencia para el prototipo pasa-bajos y $\omega$ a la frecuencia para el pasa-altos

Despejamos el $\epsilon$ como:

$\epsilon^2 = 10^{\frac{\alpha_{Máx}}{10}}-1$

In [4]:
import numpy as np
alpha_max = 1
ee = 10**(alpha_max/10)-1
e = np.sqrt(ee)
print(f'ee = {ee}')
print(f'e = {e}')

ee = 0.2589254117941673
e = 0.5088471399095875


Luego podemos hallar n (el orden del filtro) iterando en la ecuación:

$\alpha = 10 . log(1 + \epsilon^2 . \Omega_s^{2.n})$

In [6]:
Ws = 3.5
for n in range(1,6):
    alpha = 10 * np.log10(1+ee*Ws**(2*n))
    print(f'para n = {n}: alpha = {alpha}')

para n = 1: alpha = 6.203272580959268
para n = 2: alpha = 16.004827547031187
para n = 3: alpha = 26.784944183800718
para n = 4: alpha = 37.657935084281895
para n = 5: alpha = 48.53861199438661


Nos quedamos con $n = 4$ que es el menor orden que cumple con la condición de $\alpha_{Mín}$

Aplicamos una renormalización para poder trabajar el filtro con la función de aproximación de Butterworth.

$|T_{(j\Omega)}|^2=\frac{1}{1+\epsilon^2 . \Omega^{2 . n}}$

$|T_{(j\omega)}|^2=\frac{1}{1+\epsilon^2 . (\frac{\Omega_{\omega}}{\omega})^{2 . n}}$

Importante: no confundir $\Omega_\omega$ que es la norma de frecuencia con $\Omega$ que es la variable de la función transformada.

Para obtener $\omega_b$ debe considerarse $\epsilon^{\frac{1}{n}}$ para el pasa altos, a diferencia del pasabajos donde debíamos hacer $\epsilon^{\frac{-1}{n}}$.

A partir de acá, se puede tratar la función como una aproximación de Butterworth. Los polos serán 2 pares de polos complejos conjugados que quedarán sobre una circunferencia de radio unitario con una separación entre sí de $\frac{\pi}{4}$.

Para la aproximación de Butterworth, los polinomios de orden 2 tienen la forma:

$T(s) = \frac{1}{s^2 + s . \frac{1}{Q} + 1}$

Donde:

$\frac{1}{Q} = 2 . cos(\psi)$

El primer par de polos tendrá una apertura angular de $\pm\frac{\pi}{8}$ y el segundo de $\pm\frac{3}{8}.\pi$ por lo cual podemos expresar la transferencia como:

$T(s')=\frac{1}{s'^2 + s' . 2 . cos(\frac{\pi}{8}) + 1} . \frac{1}{s'^2 + s' . 2 . cos(\frac{3}{8}) + 1}$

Llamamos s' a la variable compleja correspondiente al pasa-bajos y s a la variable compleja correspondiente al pasa-altos

In [7]:
pi = np.pi
a = 2 * np.cos(pi/8)
b = 2 * np.cos(3*pi/8)

print(f'a: {a}')
print(f'b: {b}')

a: 1.8477590650225735
b: 0.7653668647301797


Expresamos la transferencia del pasa-bajos prototipo como:

$T(s')=\frac{1}{s'^2 + s' . 1,8477 + 1} . \frac{1}{s'^2 + s' . 0,7653 + 1}$

Aplicamos la transformación para pasar del pasa-bajos prototipo al pasa-altos.

$s' = \frac{1}{s}$

$T(s)=\frac{1}{(\frac{1}{s})^2 + \frac{1}{s} . 1,8477 + 1} . \frac{1}{(\frac{1}{s})^2 + \frac{1}{s} . 0,7653 + 1}$

$T(s)=\frac{s^2}{s^2 + s . 1,8477 + 1} . \frac{s^2}{s^2 + s . 0,7653 + 1}$

Importante: El pasa-altos de esta última expresión no es el objetivo, ya que sigue normalizado para Butterworth. Esta expresión de pasa-altos corresponde a una $\alpha_{Máx}$ de $3dB$ y no de $1dB$ como se nos pide. El verdadero pasa-altos que queremos lo obtenemos al final cuando desnormalizamos los componentes.

Planteamos un circuito pasivo de orden 2 con una red RLC en la que tomamos la tensión en la salida del inductor.

![circuito_rlc](img/circuito_rlc.JPG)

Su transferencia se expresa como:

$T(s) = \frac{s^2}{s^2 + s . \frac{R}{L} + \frac{1}{L.C}}$

Donde:

$\omega_0^2 = \frac{1}{L.C}$

$\frac{\omega_0}{Q} = \frac{R}{L}$

De estas 2 expresiones podemos despejar que:

$Q = \sqrt{\frac{L}{C}} . \frac{1}{R}$

Fijamos los valores $L = 1$ y $C = 1$

En la práctica, conviene fijar los valores de C ya que es más fácil ajustar en la resistencia que en el capacitor.

Con $L = 1$ y $C = 1$ entonces:

$Q = \frac{1}{R}$

$R = \frac{1}{Q}$

Para la primera etapa $R_1 = 1,8477$ y para la segunda $R_2 = 0,7653$

El circuito normalizado nos queda:

![circuito_normalizado](img/circuito_normalizado.JPG)

Al desnormalizar en frecuencia y en impedancia:

$R_1' = R_1 . \Omega_Z$

$R_2' = R_2 . \Omega_Z$

$C' = \frac{C}{\Omega_\omega . \Omega_Z}$

$L' = \frac{L . \Omega_Z}{\Omega_\omega}$

In [8]:
fp = 3500
Wz = 1000
Ww = 2*pi*fp*(e**(1/4))

R1_n = 1.8477
R2_n = 0.7653
C_n = 1
L_n = 1

R1 = R1_n * Wz
R2 = R2_n * Wz
C = C_n/(Ww*Wz)
L = (L_n*Wz)/Ww

print(f'R1={R1}')
print(f'R2={R2}')
print(f'C={C}')
print(f'L={L}')

R1=1847.6999999999998
R2=765.3
C=5.384002499547252e-08
L=0.05384002499547252


$R_1' = 1847,7\Omega$

$R_2' = 765,3\Omega$

$C' = 53,84nF$

$L' = 53,84mHy$

El circuito desnormalizado queda de la siguiente forma:

![circuito_desnormalizado](img/circuito_desnormalizado.JPG)

Al simularlo se puede observar el siguiente gráfico:

![simulacion_desnormalizada](img/simulacion_desnormalizado.JPG)